In [1]:
import brightway2 as bw

In [2]:
import bw2data as bd

In [3]:
bw.projects

Brightway2 projects manager with 14 objects:
	StepByStep
	bw2_class_2020_example_database
	bw2_class_2020_intro
	car comparison paper
	check_PBsmethod
	check_impact_electricity
	default
	double_count_electricity
	ecoinvent 3.6
	global_power_mix
	parameters - manual creation
	premise_scenarios
	simapro-ecoinvent-import
	trial-import_from_SimaPro
Use `projects.report()` to get a report on all projects.

In [4]:
bw.projects.set_current("default")

In [6]:
bw.databases

Databases dictionary with 3 object(s):
	biosphere3
	ei35_apos
	ei35_cutoff

In [16]:
rdm = bw.Database("biosphere3").random()

In [20]:
[v for v in rdm.keys()]

['categories', 'code', 'CAS number', 'name', 'database', 'unit', 'type']

In [18]:
rdm.as_dict()

{'categories': ('air', 'low population density, long-term'),
 'code': '06a4fedf-dab4-4ef1-90df-1d0ce8fe6477',
 'CAS number': '000353-59-3',
 'name': 'Methane, bromochlorodifluoro-, Halon 1211',
 'database': 'biosphere3',
 'unit': 'kilogram',
 'type': 'emission'}

In [25]:
[k for k in bw.Database("ei35_apos").random().keys()]

['comment',
 'classifications',
 'activity type',
 'activity',
 'database',
 'filename',
 'location',
 'name',
 'synonyms',
 'parameters',
 'authors',
 'type',
 'reference product',
 'flow',
 'unit',
 'production amount',
 'code']

In [31]:
# [act for act in bw.Database("ei35_apos") if "fertiliser" in act["name"]]
[act for act in bw.Database("ei35_apos") if "as N" in act["name"]]

['market for ammonium sulfate, as N' (kilogram, GLO, None),
 'market for ammonium nitrate, as N' (kilogram, GLO, None),
 'urea production, as N' (kilogram, RoW, None),
 'market for urea, as N' (kilogram, GLO, None),
 'market for nitrogen fertiliser, as N' (kilogram, GLO, None),
 'urea production, as N' (kilogram, RER, None)]

In [75]:
# What Ibrahim filtered for
[
    (act, act["reference product"])
    for act in bw.Database("ei35_cutoff")
    if "as N" in act["name"] 
    and "urea production" not in act["name"]
    or ("ammonium nitrate phosphate" in act["name"] and "as N" in act["reference product"])
]

[('market for ammonium nitrate, as N' (kilogram, GLO, None),
  'ammonium nitrate, as N'),
 ('ammonium nitrate phosphate production' (kilogram, RER, None),
  'nitrogen fertiliser, as N'),
 ('ammonium nitrate phosphate production' (kilogram, RoW, None),
  'nitrogen fertiliser, as N'),
 ('market for ammonium sulfate, as N' (kilogram, GLO, None),
  'ammonium sulfate, as N'),
 ('market for urea, as N' (kilogram, GLO, None), 'urea, as N'),
 ('market for nitrogen fertiliser, as N' (kilogram, GLO, None),
  'nitrogen fertiliser, as N')]

In [76]:
# [
#     act["reference product"]
#     for act in bw.Database("ei35_cutoff")
#     if "ammonium nitrate phosphate" in act["name"]
# ]

In [48]:
[act["classifications"] for act in bw.Database("ei35_cutoff") if "ammonium nitrate phosphate" in act["name"]][0]

[('EcoSpold01Categories',
  'agricultural means of production/mineral fertiliser'),
 ('ISIC rev.4 ecoinvent',
  '2012:Manufacture of fertilizers and nitrogen compounds'),
 ('CPC', '3462: Mineral or chemical fertilizers, phosphatic')]

In [54]:
isic = []
cpc = []
ecospold = []
for act in bw.Database("ei35_cutoff"):
    for classification in act["classifications"]:
        if "isic" in classification[0].lower():
            isic.append(classification)
        elif "ecospold" in classification[0].lower():
            ecospold.append(classification)
        elif "cpc" in classification[0].lower():
            cpc.append(classification)
isic = list(set(isic))
cpc = list(set(cpc))
ecospold = list(set(ecospold))

In [99]:
# sorted([e[1] for e in ecospold])
# sorted([e[1] for e in cpc])
# sorted([e[1] for e in isic])

In [136]:
ACTIVITIES = []
for act in bw.Database("ei35_cutoff"):
    for classification in act["classifications"]:
        if "cpc" in classification[0].lower() and classification[1].startswith("3461"):
            if "market" not in act["name"]:
                ACTIVITIES.append(act)

In [137]:
print(len(ACTIVITIES))
ACTIVITIES

44


['nutrient supply from manure, liquid, swine' (kilogram, GLO, None),
 'ammonium sulfate production' (kilogram, RER, None),
 'nutrient supply from potassium nitrate' (kilogram, GLO, None),
 'citric acid production' (kilogram, RNA, None),
 'treatment, sludge from pulp and paper production, landfarming' (kilogram, CA-QC, None),
 'ammonium nitrate production' (kilogram, RER, None),
 'ammonium nitrate phosphate production' (kilogram, RoW, None),
 'smelting and refining of nickel ore' (kilogram, GLO, None),
 'nutrient supply from manure, solid, cattle' (kilogram, GLO, None),
 'nutrient supply from vinasse, from fermentation of sweet sorghum' (kilogram, GLO, None),
 'diammonium phosphate production' (kilogram, RoW, None),
 'nutrient supply from coconut husk' (kilogram, GLO, None),
 'calcium nitrate production' (kilogram, RoW, None),
 'monoammonium phosphate production' (kilogram, RoW, None),
 'nutrient supply from vinasse, from fermentation of sugar beet' (kilogram, GLO, None),
 'urea product

In [140]:
# [exc for exc in [a for a in ACTIVITIES if "market" in a["name"]][0].exchanges()]

In [141]:
[
    (act["reference product"], act["classifications"])
    for act in bw.Database("ei35_cutoff")
    # if "nickel mine operation, sulfidic ore" in act["name"] and "GLO" in act["location"]
    # if 'smelting and refining of nickel ore' in act["name"] and "GLO" in act["location"]
    # if "market for ammonium nitrate, as N" in act["name"] and "GLO" in act["location"]
    if "heat production, straw, at furnace 300kW" in act["name"] and "GLO" in act["location"]
]

[('heat, district or industrial, other than natural gas',
  [('EcoSpold01Categories', 'wood energy/heating systems'),
   ('ISIC rev.4 ecoinvent', '3530:Steam and air conditioning supply'),
   ('CPC', '17300: Steam and hot water')]),
 ('phosphate fertiliser, as P2O5',
  [('EcoSpold01Categories', 'wood energy/heating systems'),
   ('ISIC rev.4 ecoinvent', '3530:Steam and air conditioning supply'),
   ('CPC', '3462: Mineral or chemical fertilizers, phosphatic')]),
 ('nitrogen fertiliser, as N',
  [('EcoSpold01Categories', 'wood energy/heating systems'),
   ('ISIC rev.4 ecoinvent', '3530:Steam and air conditioning supply'),
   ('CPC', '3461: Mineral or chemical fertilizers, nitrogenous')])]

In [ ]:
[
    t
    for t in [
        act
        for act in bw.Database("ei35_cutoff")
        if "heat production, straw, at furnace 300kW" in act["name"]
        and "GLO" in act["location"]
    ][0].technosphere()
]

In [142]:
sorted([(a["reference product"], a["name"]) for a in ACTIVITIES])

[('ammonium nitrate, as N', 'ammonium nitrate production'),
 ('ammonium nitrate, as N', 'ammonium nitrate production'),
 ('ammonium nitrate, as N', 'calcium nitrate production'),
 ('ammonium nitrate, as N', 'calcium nitrate production'),
 ('ammonium nitrate, as N', 'citric acid production'),
 ('ammonium nitrate, as N', 'citric acid production'),
 ('ammonium nitrate, as N', 'citric acid production'),
 ('ammonium nitrate, as N', 'citric acid production'),
 ('ammonium sulfate, as N', 'ammonium sulfate production'),
 ('ammonium sulfate, as N', 'ammonium sulfate production'),
 ('ammonium sulfate, as N', 'nickel mine operation, sulfidic ore'),
 ('ammonium sulfate, as N', 'smelting and refining of nickel ore'),
 ('nitrogen fertiliser, as N', 'ammonium nitrate phosphate production'),
 ('nitrogen fertiliser, as N', 'ammonium nitrate phosphate production'),
 ('nitrogen fertiliser, as N', 'calcium ammonium nitrate production'),
 ('nitrogen fertiliser, as N', 'calcium ammonium nitrate production')

In [404]:
# del bw.databases["A_technosphere_flows"]
bw.databases

Databases dictionary with 3 object(s):
	biosphere3
	ei35_apos
	ei35_cutoff

In [405]:
# temporal

# del bw.databases["ei35_cutoff"]

# PATH_TO_EI_DATASETS = r"C:\Users\ViteksPC\Documents\00-ETH_projects\18-BW2Hub\01.Data\Ecoinvent\v3.5\ecoinvent 3.5_cutoff_ecoSpold02\datasets"
# ei = bw.SingleOutputEcospold2Importer(PATH_TO_EI_DATASETS, "ei35_cutoff")
# ei
# ei.apply_strategies()
# ei.statistics()
# ei.write_database()
# del ei

In [406]:
# write new database
bw.Database("A_technosphere_flows").write(
    {
        ("A_technosphere_flows", "n-fert"): {  # (db name, code)
            "name": "nitrogen fertilizer",
            "unit": "kilogram",
            "type": "inventory flow",
            "categories": ("inventory",),
        }
    }
)

Writing activities to SQLite3 database:
0% [#] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 05/15/2022 20:52:08
  Finished: 05/15/2022 20:52:08
  Total time elapsed: 00:00:00
  CPU %: 0.00
  Memory %: 7.43


In [407]:
[a for a in bw.Database("A_technosphere_flows")][0].as_dict()

{'name': 'nitrogen fertilizer',
 'unit': 'kilogram',
 'type': 'inventory flow',
 'categories': ('inventory',),
 'database': 'A_technosphere_flows',
 'code': 'n-fert'}

In [408]:
bw.databases

Databases dictionary with 4 object(s):
	A_technosphere_flows
	biosphere3
	ei35_apos
	ei35_cutoff

In [437]:
# create exchanges for "nitrogen fertilizer" if they don't exist already
def is_exchange(exc):
    return (
        "nitrogen fertilizer" == bw.get_activity(exc["input"])["name"]
        and ("inventory",) == bw.get_activity(exc["input"])["categories"]
    )


print("Updating activities that produce nitrogen fertilizer:")
for act in ACTIVITIES:
    if not list(filter(is_exchange, [exc for exc in act.biosphere()])):
        act.new_exchange(
            **{
                "name": "nitrogen fertilizer",
                "input": ("A_technosphere_flows", "n-fert"),
                "type": "biosphere",
                "amount": 1,
                "unit": "kilogram",
            }
        ).save()
        print(f"__ 1 new exchange has been added to {act}.")
    else:
        print(f"xx {act} already contains the necessary exchanges.")

Updating activities that produce nitrogen fertilizer:
__ 1 new exchange has been added to 'nutrient supply from manure, liquid, swine' (kilogram, GLO, None).
__ 1 new exchange has been added to 'ammonium sulfate production' (kilogram, RER, None).
xx 'nutrient supply from potassium nitrate' (kilogram, GLO, None) already contains the necessary exchanges.
xx 'citric acid production' (kilogram, RNA, None) already contains the necessary exchanges.
xx 'treatment, sludge from pulp and paper production, landfarming' (kilogram, CA-QC, None) already contains the necessary exchanges.
xx 'ammonium nitrate production' (kilogram, RER, None) already contains the necessary exchanges.
xx 'ammonium nitrate phosphate production' (kilogram, RoW, None) already contains the necessary exchanges.
xx 'smelting and refining of nickel ore' (kilogram, GLO, None) already contains the necessary exchanges.
xx 'nutrient supply from manure, solid, cattle' (kilogram, GLO, None) already contains the necessary exchanges.

In [439]:
a = ACTIVITIES[0]
[exc for exc in a.biosphere()]#[0].delete()

[Exchange: 1 kilogram 'nitrogen fertilizer' (kilogram, None, ('inventory',)) to 'nutrient supply from manure, liquid, swine' (kilogram, GLO, None)>]

In [442]:
# for exc in a.exchanges():
#     if exc["input"] == ("A_technosphere_flows", "n-fert"):
#         print(bw.get_activity(exc["input"]))

In [444]:
# a.as_dict()

In [517]:
m = bw.Method(("Nflows", "N fertilizer"))
m.register(unit="Tg N")
m.write([(("A_technosphere_flows", "n-fert"), 1e-9)])

In [518]:
[m for m in bw.methods if "Nflows" in m]

[('Nflows', 'N fertilizer')]

In [520]:
bw.Method(('Nflows', 'N fertilizer')).metadata

{'unit': 'Tg N',
 'abbreviation': 'nflowsn.c6f1c37e1905f51f453ecc3274358fea',
 'num_cfs': 1}

In [521]:
[m for m in bw.methods if "AESA" in str(m) and "nitrogen" in str(m)][0]

('AESA (PBs-LCIA)', '0.8.6', 'biogeochemical flows', 'nitrogen')

In [522]:
bw.Method(('AESA (PBs-LCIA)', '0.8.6', 'biogeochemical flows', 'nitrogen')).metadata

{'description': '{"overview": "industrial and intentional biological fixation of N", "authors": "Ryberg, M. W.; Owsianiak, M.; Richardson, K.; Hauschild, M. Z.", "doi": "https://doi.org/10.1016/j.ecolind.2017.12.065", "current_version": "v0.8.6", "changelog": "Find changelog here: https://github.com/vtulus/AESAmethods/blob/master/CHANGELOG.md", "implemented_by": "Tulus, V."}',
 'filename': 'aesa_BiogeochemicalFlows_N.xlsx',
 'unit': 'Tg N',
 'abbreviation': 'aesa-pbs-lcia086bn.ddc0d04c8b8df5fa26932f08462a9d8c',
 'num_cfs': 2}

In [475]:
b = [act for act in bw.Database("ei35_cutoff") if "market for nitrogen fertiliser, as N" in act["name"]][0]
b

'market for nitrogen fertiliser, as N' (kilogram, GLO, None)

In [476]:
methanol = [act for act in bw.Database("ei35_cutoff") if "market for methanol" == act["name"]][0]
methanol

'market for methanol' (kilogram, GLO, None)

In [499]:
elec = [act for act in bw.Database("ei35_cutoff") if "market group for electricity, medium" in act["name"] and "GLO" in act["location"]][0]
elec

'market group for electricity, medium voltage' (kilowatt hour, GLO, None)

In [509]:
lca = bw.LCA(
     {b: 1}, ('Nflows', 'N fertilizer')
    # {elec: 1}, ('Nflows', 'N fertilizer')
    # {methanol: 1}, ('Nflows', 'N fertilizer')
)
lca.lci()
lca.lcia()
lca.score

1.0011173644473008e-09

In [510]:
lca1 = bw.LCA(
    {b: 1}, ("AESA (PBs-LCIA)", "0.8.6", "biogeochemical flows", "nitrogen")
    # {elec: 1}, ("AESA (PBs-LCIA)", "0.8.6", "biogeochemical flows", "nitrogen")
    # {methanol: 1}, ("AESA (PBs-LCIA)", "0.8.6", "biogeochemical flows", "nitrogen")
)
lca1.lci()
lca1.lcia()
lca1.score

1.0729868277628887e-11

In [511]:
(lca1.score-lca.score)/lca1.score

-92.30192492060398

In [224]:
bw.Method(lca.method).metadata

{'unit': 'kilogram',
 'abbreviation': 'nflowsn.c6f1c37e1905f51f453ecc3274358fea',
 'num_cfs': 1}

In [217]:
bw.Method(('Nflows', 'N fertilizer')).name

('Nflows', 'N fertilizer')

In [184]:
[flow for flow in bw.Database("biosphere3")][0].as_dict()

{'categories': ('water', 'surface water'),
 'code': '88eaa159-4194-49eb-9905-9b937e644fb8',
 'CAS number': '016872-11-0',
 'name': 'Fluoboric acid',
 'database': 'biosphere3',
 'unit': 'kilogram',
 'type': 'emission'}

In [508]:
#TODO: don't forget to delete Nflows method after trials...